In [31]:
import re
import tarfile
from functools import reduce
import numpy as np

from keras.utils.data_utils import get_file
from keras.layers.embeddings import Embedding
from keras import layers
from keras.layers import Input, Embedding, Dropout, RepeatVector, Dense, recurrent, Merge
from keras.models import Model, Sequential
from keras.preprocessing.sequence import pad_sequences

In [2]:
RNN = recurrent.LSTM
EMBED_HIDDEN_SIZE = 50
SENT_HIDDEN_SIZE = 100
QUERY_HIDDEN_SIZE = 100
BATCH_SIZE = 32
EPOCHS = 40
print('RNN / Embed / Sent / Query = {}, {}, {}, {}'.format(RNN,
                                                           EMBED_HIDDEN_SIZE,
                                                           SENT_HIDDEN_SIZE,
                                                           QUERY_HIDDEN_SIZE))

RNN / Embed / Sent / Query = <class 'keras.layers.recurrent.LSTM'>, 50, 100, 100


In [3]:
try:
    path = get_file('babi-tasks-v1-2.tar.gz', origin='https://s3.amazonaws.com/text-datasets/babi_tasks_1-20_v1-2.tar.gz')
except:
    print('Error downloading dataset.')

In [4]:
path

'/Users/koichiro.mori/.keras/datasets/babi-tasks-v1-2.tar.gz'

In [5]:
tar = tarfile.open(path)

In [6]:
challenge = 'tasks_1-20_v1-2/en/qa2_two-supporting-facts_{}.txt'

In [7]:
def tokenize(sent):
    return [x.strip() for x in re.split('(\W+)?', sent) if x.strip()]

def parse_stories(lines, only_supporting=False):
    data = []
    story = []
    for line in lines:
        line = line.decode('utf-8').strip()
        # 最初の空白のみsplitする
        nid, line = line.split(' ', 1)
        nid = int(nid)
        if nid == 1:
            story = []
        if '\t' in line:  # 質問、回答、根拠が書いてある行
            q, a, supporting = line.split('\t')
            q = tokenize(q)  # 単語に分割
            substory = None
            if only_supporting:
                # 根拠となる文のみを取得
                supporting = map(int, supporting.split())
                substory = [story[i - 1] for i in supporting]
            else:
                # 全文を取得
                substory = [x for x in story if x]
            data.append((substory, q, a))
            story.append('')  # 質問は空の文章を追加しておく
        else:
            sent = tokenize(line)
            story.append(sent)
    return data

def get_stories(f, only_supporting=False, max_length=None):
    # max_lengthが指定されたらmax_lengthより長いストーリーは無視される
    # data[storyID] = (substory, quetion, answer)
    data = parse_stories(f.readlines(), only_supporting=only_supporting)
    # 単語を結合して1つのlistにする関数
    flatten = lambda data: reduce(lambda x, y: x + y, data)
    # substoryを単語リストにする
    data = [(flatten(story), q, answer) for story, q, answer in data if not max_length or len(flatten(story)) < max_length]
    return data

In [8]:
# train[storyID] = ([story word list], [question word list], answer word)
train = get_stories(tar.extractfile(challenge.format('train')))
test = get_stories(tar.extractfile(challenge.format('test')))
print(len(train), len(test))
print(train[0])

/Users/koichiro.mori/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/re.py:203: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


1000 1000
(['Mary', 'moved', 'to', 'the', 'bathroom', '.', 'Sandra', 'journeyed', 'to', 'the', 'bedroom', '.', 'Mary', 'got', 'the', 'football', 'there', '.', 'John', 'went', 'to', 'the', 'kitchen', '.', 'Mary', 'went', 'back', 'to', 'the', 'kitchen', '.', 'Mary', 'went', 'back', 'to', 'the', 'garden', '.'], ['Where', 'is', 'the', 'football', '?'], 'garden')


In [9]:
# 訓練データとテストデータに出現する語彙集合
vocab = set()
for story, q, answer in train + test:
    vocab |= set(story + q + [answer])
vocab = sorted(vocab)
print(len(vocab), vocab[:100])

35 ['.', '?', 'Daniel', 'John', 'Mary', 'Sandra', 'Where', 'apple', 'back', 'bathroom', 'bedroom', 'discarded', 'down', 'dropped', 'football', 'garden', 'got', 'grabbed', 'hallway', 'is', 'journeyed', 'kitchen', 'left', 'milk', 'moved', 'office', 'picked', 'put', 'the', 'there', 'to', 'took', 'travelled', 'up', 'went']


In [10]:
# 0はpad用にあけておく
# 単語 => 単語IDへの写像
vocab_size = len(vocab) + 1
word_idx = dict((c, i + 1) for i, c in enumerate(vocab))
print(word_idx)

# ストーリーと質問の最大長を計算
story_maxlen = max(map(len, (x for x, _, _ in train + test)))
query_maxlen = max(map(len, (x for _, x, _ in train + test)))
print('story_maxlen', story_maxlen)
print('query_maxlen', query_maxlen)

{'to': 31, 'down': 13, 'bathroom': 10, 'garden': 16, 'John': 4, 'grabbed': 18, 'Where': 7, 'Daniel': 3, 'apple': 8, 'kitchen': 22, 'back': 9, 'took': 32, '.': 1, 'hallway': 19, 'discarded': 12, 'bedroom': 11, 'there': 30, 'put': 28, 'journeyed': 21, 'office': 26, 'dropped': 14, 'picked': 27, 'went': 35, 'is': 20, 'Sandra': 6, 'moved': 25, 'football': 15, '?': 2, 'milk': 24, 'left': 23, 'Mary': 5, 'travelled': 33, 'up': 34, 'the': 29, 'got': 17}
story_maxlen 552
query_maxlen 5


In [11]:
def vectorize_stories(data, word_idx, story_maxlen, query_maxlen):
    xs = []
    xqs = []
    ys = []
    for story, query, answer in data:
        # storyとqueryは単語ID系列で作成
        x = [word_idx[w] for w in story]
        xq = [word_idx[w] for w in query]
        # 回答はvocabの長さのベクトル（one-hot-encoding)
        y = np.zeros(len(word_idx) + 1)
        y[word_idx[answer]] = 1
        xs.append(x)
        xqs.append(xq)
        ys.append(y)
    xs = pad_sequences(xs, maxlen=story_maxlen)
    xqs = pad_sequences(xqs, maxlen=query_maxlen)
    ys = np.array(ys)
    return xs, xqs, ys

In [12]:
idx_word = {y:x for x,y in word_idx.items()}
print(idx_word)

{1: '.', 2: '?', 3: 'Daniel', 4: 'John', 5: 'Mary', 6: 'Sandra', 7: 'Where', 8: 'apple', 9: 'back', 10: 'bathroom', 11: 'bedroom', 12: 'discarded', 13: 'down', 14: 'dropped', 15: 'football', 16: 'garden', 17: 'got', 18: 'grabbed', 19: 'hallway', 20: 'is', 21: 'journeyed', 22: 'kitchen', 23: 'left', 24: 'milk', 25: 'moved', 26: 'office', 27: 'picked', 28: 'put', 29: 'the', 30: 'there', 31: 'to', 32: 'took', 33: 'travelled', 34: 'up', 35: 'went'}


In [15]:
# 文章をベクトル化
x, xq, y = vectorize_stories(train, word_idx, story_maxlen, query_maxlen)
tx, txq, ty = vectorize_stories(test, word_idx, story_maxlen, query_maxlen)
print(len(x))
print(len(xq))
print(len(y))
print(x[0])
print(xq[0], [idx_word[i] for i in xq[0]])
print(y[0], idx_word[np.argmax(y[0])])

1000
1000
1000
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 

In [16]:
pad_sequences([[1,2,3,4,5], [1,2,3], [8]], 10)

array([[0, 0, 0, 0, 0, 1, 2, 3, 4, 5],
       [0, 0, 0, 0, 0, 0, 0, 1, 2, 3],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 8]], dtype=int32)

In [17]:
print('vocab = {} {}'.format(len(vocab), vocab))
print('x.shape = {}'.format(x.shape))
print('xq.shape = {}'.format(xq.shape))
print('y.shape = {}'.format(y.shape))
print('story_maxlen, query_maxlen = {} {}'.format(story_maxlen, query_maxlen))

vocab = 35 ['.', '?', 'Daniel', 'John', 'Mary', 'Sandra', 'Where', 'apple', 'back', 'bathroom', 'bedroom', 'discarded', 'down', 'dropped', 'football', 'garden', 'got', 'grabbed', 'hallway', 'is', 'journeyed', 'kitchen', 'left', 'milk', 'moved', 'office', 'picked', 'put', 'the', 'there', 'to', 'took', 'travelled', 'up', 'went']
x.shape = (1000, 552)
xq.shape = (1000, 5)
y.shape = (1000, 36)
story_maxlen, query_maxlen = 552 5


In [18]:
print('Build model ...')

sentence = layers.Input(shape=(story_maxlen, ), dtype='int32')
encoded_sentence = Embedding(vocab_size, EMBED_HIDDEN_SIZE)(sentence)
encode_sentence = Dropout(0.3)(encoded_sentence)

question = layers.Input(shape=(query_maxlen, ), dtype='int32')
encoded_question = Embedding(vocab_size, EMBED_HIDDEN_SIZE)(question)
encoded_question = Dropout(0.3)(encoded_question)
encoded_question = RNN(EMBED_HIDDEN_SIZE)(encoded_question)
encoded_question = RepeatVector(story_maxlen)(encoded_question)

merged = layers.add([encoded_sentence, encoded_question])
merged = RNN(EMBED_HIDDEN_SIZE)(merged)
merged = Dropout(0.3)(merged)
preds = Dense(vocab_size, activation='softmax')(merged)

model = Model([sentence, question], preds)

Build model ...


In [19]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 5)             0                                            
____________________________________________________________________________________________________
embedding_2 (Embedding)          (None, 5, 50)         1800        input_2[0][0]                    
____________________________________________________________________________________________________
dropout_2 (Dropout)              (None, 5, 50)         0           embedding_2[0][0]                
____________________________________________________________________________________________________
input_1 (InputLayer)             (None, 552)           0                                            
___________________________________________________________________________________________

In [22]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [23]:
y.shape

(1000, 36)

In [24]:
print('Training')
model.fit([x, xq], y,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_split=0.05)

Training
Train on 950 samples, validate on 50 samples
Epoch 1/40
950/950 [==============================] - 50s - loss: 1.8081 - acc: 0.2116 - val_loss: 1.7693 - val_acc: 0.1400
Epoch 2/40
950/950 [==============================] - 49s - loss: 1.7926 - acc: 0.2242 - val_loss: 1.7677 - val_acc: 0.1800
Epoch 3/40
950/950 [==============================] - 48s - loss: 1.7784 - acc: 0.2000 - val_loss: 1.7680 - val_acc: 0.2000
Epoch 4/40
950/950 [==============================] - 49s - loss: 1.7474 - acc: 0.2516 - val_loss: 1.7511 - val_acc: 0.2200
Epoch 5/40
950/950 [==============================] - 49s - loss: 1.7546 - acc: 0.2326 - val_loss: 1.7563 - val_acc: 0.3000
Epoch 6/40
950/950 [==============================] - 49s - loss: 1.7526 - acc: 0.2200 - val_loss: 1.7494 - val_acc: 0.3000
Epoch 7/40
950/950 [==============================] - 49s - loss: 1.7438 - acc: 0.2442 - val_loss: 1.7334 - val_acc: 0.3600
Epoch 8/40
950/950 [==============================] - 49s - loss: 1.7165 - acc

In [25]:
loss, acc = model.evaluate([tx, txq], ty,
                           batch_size=BATCH_SIZE)
print('Test loss / test accuracy = {:.4f} / {:.4f}'.format(loss, acc))

1000/1000 [==============================] - 10s    
Test loss / test accuracy = 1.5878 / 0.3660


## architecture2

In [34]:
# sentence = layers.Input(shape=(story_maxlen, ), dtype='int32')
# encoded_sentence = Embedding(vocab_size, EMBED_HIDDEN_SIZE)(sentence)
# encode_sentence = Dropout(0.3)(encoded_sentence)

# question = layers.Input(shape=(query_maxlen, ), dtype='int32')
# encoded_question = Embedding(vocab_size, EMBED_HIDDEN_SIZE)(question)
# encoded_question = Dropout(0.3)(encoded_question)
# encoded_question = RNN(EMBED_HIDDEN_SIZE)(encoded_question)
# encoded_question = RepeatVector(story_maxlen)(encoded_question)

# merged = layers.add([encoded_sentence, encoded_question])
# merged = RNN(EMBED_HIDDEN_SIZE)(merged)
# merged = Dropout(0.3)(merged)
# preds = Dense(vocab_size, activation='softmax')(merged)

# model = Model([sentence, question], preds)

sentence = layers.Input(shape=(story_maxlen, ), dtype='int32')
encoded_sentence = Embedding(vocab_size, EMBED_HIDDEN_SIZE)(sentence)
encoded_sentence = RNN(EMBED_HIDDEN_SIZE)(encoded_sentence)

question = layers.Input(shape=(query_maxlen, ), dtype='int32')
encoded_question = Embedding(vocab_size, EMBED_HIDDEN_SIZE)(question)
encoded_question = RNN(EMBED_HIDDEN_SIZE)(encoded_question)

concated = layers.concatenate([encoded_sentence, encoded_question])
preds = Dense(vocab_size, activation='softmax')(concated)

model = Model([sentence, question], preds)

In [35]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_5 (InputLayer)             (None, 552)           0                                            
____________________________________________________________________________________________________
input_6 (InputLayer)             (None, 5)             0                                            
____________________________________________________________________________________________________
embedding_7 (Embedding)          (None, 552, 50)       1800        input_5[0][0]                    
____________________________________________________________________________________________________
embedding_8 (Embedding)          (None, 5, 50)         1800        input_6[0][0]                    
___________________________________________________________________________________________

In [36]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [37]:
print('Training')
model.fit([x, xq], y,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_split=0.05)

Training
Train on 950 samples, validate on 50 samples
Epoch 1/40
950/950 [==============================] - 48s - loss: 3.0932 - acc: 0.1611 - val_loss: 2.1091 - val_acc: 0.1400
Epoch 2/40
950/950 [==============================] - 45s - loss: 1.8625 - acc: 0.1895 - val_loss: 1.7594 - val_acc: 0.3000
Epoch 3/40
950/950 [==============================] - 46s - loss: 1.7928 - acc: 0.2011 - val_loss: 1.8598 - val_acc: 0.0600
Epoch 4/40
950/950 [==============================] - 45s - loss: 1.7923 - acc: 0.2074 - val_loss: 1.8603 - val_acc: 0.0600
Epoch 5/40
950/950 [==============================] - 45s - loss: 1.7997 - acc: 0.1937 - val_loss: 1.7662 - val_acc: 0.3000
Epoch 6/40
950/950 [==============================] - 45s - loss: 1.7893 - acc: 0.1947 - val_loss: 1.8175 - val_acc: 0.0600
Epoch 7/40
950/950 [==============================] - 45s - loss: 1.7947 - acc: 0.2042 - val_loss: 1.8240 - val_acc: 0.0600
Epoch 8/40
950/950 [==============================] - 45s - loss: 1.7877 - acc